<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-395/477 Redes Neuronales Artificiales I-2018 </h1>

<H3 align='center'> Tarea 0 - Introducción a Redes Neuronales </H3>
<div align='center'>
###  Felipe González - 201273534-3 
### Ignacio Tampe - 201573514-k 
</div>
<hr style="height:2px;border:none"/>

## 1. Back-propagation (BP) from *Scratch*


In [1]:
import numpy as np #se importa numpy como np
import random

In [2]:
class RedNeuronal:
    # num_entrada: dimension de los vectores de entrada, en iris: 4
    # tam_capas: lista de cantidad de neuronas por cada capa interna, en iris: [32, 16]
    # num_salida: dimension de los vectores de salida, en iris: 3
    def __init__(self, num_entrada, tam_capas, num_salida, act_inter, cost_func):
        self.num_entrada = num_entrada
        self.tam_capas = tam_capas
        self.num_salida = num_salida
        self.capas = []
        # iniciar todas las capas inicial - ocultas - final
        self.capas.append(CapaNeuronal(num_entrada, act_inter))
        for i in range(len(tam_capas)):
            self.capas.append(CapaNeuronal(tam_capas[i], act_inter, self.capas[i].tam_capa))
        self.capas.append(CapaNeuronal(num_salida, "softmax", self.capas[len(self.capas)-1].tam_capa))
        self.cant_capas = len(self.capas)
        if cost_func == "crossentropy":
            self.cost = self.crossentropy
            self.cost_derivative = self.crossentropy_derivative
        elif cost_func == "mse":
            self.cost = self.mse
            self.cost_derivative = self.mse_derivative
        
    def crossentropy(self, prediccion, esperado, epsilon=1e-12): #El epsilon evita log(0)
        prediccion = np.clip(prediccion, epsilon, 1. - epsilon)
        N = prediccion.shape[0]
        ce = -np.sum(np.sum(esperado*np.log(prediccion+1e-9)))/N
        return ce
    
    def crossentropy_derivative(self, prediccion, esperado):
        return (esperado-1/prediccion+(1+esperado)*(1/(1-prediccion)))  
    
    def mse(self, prediccion, esperado):
        return np.dot(prediccion-esperado, prediccion-esperado)
    
    def mse_derivative(self, prediccion, esperado):
        return prediccion-esperado
    
    def train(self, entrada, salida, ciclos, tasa):
        # repetir para todas 
        last = self.cant_capas-1 # indice capa de salida
        result = zip(entrada,salida)
        resultList = list(result)
        for ciclo in range(ciclos):
            # Comienza 1 epoch
            random.shuffle(resultList)
            errores = []
            for input_v, output in resultList:
                
                self.capas[0].set_capa(input_v, input_v) #Poblar capa inicial
                # Feed forward 
                for j in range(1, len(self.capas)):
                    self.capas[j].feed_forward(self.capas[j-1])
                
                # evaluar error final (delta función costo)
                error_epoch = self.cost(self.capas[last].get_activaciones(), output)
                errores.append(error_epoch)
                ce_derivate = self.cost_derivative(self.capas[last].get_activaciones(), np.transpose(output))
                delta = ce_derivate * self.capas[last].squash_derivative(self.capas[last].get_vector_z())
                
                #Backward pass capa final
                self.capas[last].actualizar_bias(delta, tasa)
                self.capas[last].actualizar_pesos(np.outer(self.capas[last-1].get_activaciones(), delta), tasa)
                #print(self.capas[last].get_pesos())
                #Backward pass general
                for l in range(2, self.cant_capas):
                    z = self.capas[-l].get_vector_z()
                    act_prime = self.capas[-l].squash_derivative(z)
                    delta = np.dot(self.capas[-l+1].get_pesos(), delta.transpose()) * act_prime
                    self.capas[-l].actualizar_bias(delta, tasa)
                    self.capas[-l].actualizar_pesos(np.outer(self.capas[-l-1].get_activaciones(),delta), tasa)
                    
            print("Error del epoch",ciclo,":", str(np.round(np.mean(errores),7)))
        return errores

            
    def test(self, entrada, salida):
        # repetir para todas 
        last = self.cant_capas-1 # indice capa de salida
        result = zip(entrada,salida)
        resultList = list(result)
        errores = []
        for input_v, output in resultList:
            self.capas[0].set_capa(input_v, input_v) #Poblar capa inicial
            # Feed forward 
            for j in range(1, len(self.capas)):
                self.capas[j].feed_forward(self.capas[j-1])

            # evaluar error final (delta función costo)
            error_epoch = self.crossentropy(self.capas[last].get_activaciones(), output)
            errores.append(error_epoch)
            print(self.capas[last].get_activaciones(), output)
        print("Error del test:", str(np.round(np.mean(errores),7)))


In [3]:
class CapaNeuronal:
    def __init__(self, tam_capa, tipo, tam_anterior = 0):
        self.tam_capa = tam_capa
        self.tam_anterior = tam_anterior
        self.bias = 1
        self.neuronas = [Neurona(tam_anterior) for i in range(tam_capa)] # lista de neuronas de la capa
        if tipo == "sigmoid":
            self.squash = self.sigmoid
            self.squash_derivative = self.sigmoid_derivative
        elif tipo == "softmax":
            self.squash = self.softmax
            self.squash_derivative = self.softmax_derivative
        elif tipo == "tanh":
            self.squash = self.tanh
            self.squash_derivative = self.tanh_derivative
        elif tipo == "relu":
            self.squash = self.relu
            self.squash_derivative = self.relu_derivative
       
    def sigmoid(self, x):
        return np.array([np.where(i > 0, 1. / (1. + np.exp(-i)), np.exp(i) / (np.exp(i) + np.exp(0))) for i in x])
    
    def sigmoid_derivative(self, x):
        return np.multiply(self.sigmoid(x),(1-self.sigmoid(x)))
    
    def tanh(self, x):
        return np.arctan(x)
    
    def tanh_derivative(self, x):
        return np.power(np.cos(x),2)
        
    def relu(self, x):
        x= np.array(x)
        return np.maximum(x, 0, x)
    
    def relu_derivative(self, x):
        x = np.array(x)
        return (x > 0).astype(int)
    
    def softmax(self, x):
        x = np.clip(x, -350, 350)
        return np.exp(x) / np.sum(np.exp(x), axis=0)
    
    def softmax_derivative(self, x):
        x = np.clip(x, -350, 350)
        res = []
        for i in x:
            res.append(np.exp(i)*(np.sum(np.exp(x), axis=0)-np.exp(i))/(np.sum(np.exp(x), axis=0)**2))
        #print(x)
        return np.array(res)
    
    def get_activaciones(self):
        return np.array([neurona.activacion for neurona in self.neuronas])
    
    def get_vector_z(self):
        return [neurona.z for neurona in self.neuronas]
    
    
    def feed_forward(self, capa_anterior):
        pre_squash = []
        activaciones = capa_anterior.get_activaciones()
        vector_z = capa_anterior.get_vector_z()
        for neurona in self.neuronas:
            pesos = neurona.pesos
            pre_squash.append(np.dot(pesos, activaciones) + neurona.bias)
        self.set_capa(self.squash(pre_squash), pre_squash)
        
   
    def get_pesos(self):
        return np.transpose(np.array([n.pesos for n in self.neuronas]))

    def actualizar_pesos(self, matriz, rate):
        pesos = np.transpose(np.array([n.pesos for n in self.neuronas]))
        pesos -= rate*matriz #SE PUSO SUMA PORQUE BAJABA EL ERROR DEL EPOCH
        for i in range(len(self.neuronas)):
            self.neuronas[i].pesos = pesos[:,i]
    
    def actualizar_bias(self, vector_b, rate):
        biases = np.array([n.bias for n in self.neuronas])
        biases -= rate*vector_b #SE PUSO SUMA PORQUE BAJABA EL ERROR DEL EPOCH
        for i in range(len(self.neuronas)):
            self.neuronas[i].bias = biases[i]
    
    #Funcion para poblar datos de entrada a la red.
    def set_capa(self, activaciones, vector_z):
        for i in range(len(self.neuronas)):
            self.neuronas[i].set_activacion(activaciones[i], vector_z[i])
            
    def mostrar_capa(self):
        for neurona in self.neuronas:
            print(neurona, end="\t")
        print("\n-----------------------------")

        
class Neurona:
    def __init__(self, tam_capa_anterior):
        self.pesos = np.random.rand(tam_capa_anterior) # genera una lista de numeros aleatorios de tamaño "tam_capa_anterior"
        self.bias = np.random.rand()
    def set_activacion(self, valor_a, valor_z):
        self.activacion = valor_a
        self.z = valor_z
        
    def __str__(self):
        return "Activación: "+str(self.activacion)

### ¡Es hora de probar!

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import keras
x, y = load_iris(return_X_y=True)
scaler = StandardScaler().fit(x)
x = scaler.transform(x) 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
y_onehot_train = keras.utils.to_categorical(y_train)
y_onehot_test = keras.utils.to_categorical(y_test)

#transform target to one hot vector
nn = RedNeuronal(4, [32, 16], 3, "sigmoid", "mse")
nn.train(x_train, y_onehot_train, 100, 0.1)
print("TESTING")
nn.test(x_test, y_onehot_test)

Error del epoch 0 : 0.7099084
Error del epoch 1 : 0.7194283
Error del epoch 2 : 0.6940541
Error del epoch 3 : 0.7020013
Error del epoch 4 : 0.7294889
Error del epoch 5 : 0.7103572
Error del epoch 6 : 0.7224873
Error del epoch 7 : 0.721788
Error del epoch 8 : 0.7189185
Error del epoch 9 : 0.7148753
Error del epoch 10 : 0.7168664
Error del epoch 11 : 0.6955322
Error del epoch 12 : 0.7210869
Error del epoch 13 : 0.7115522
Error del epoch 14 : 0.7189777
Error del epoch 15 : 0.7163099
Error del epoch 16 : 0.7185008
Error del epoch 17 : 0.7105343
Error del epoch 18 : 0.7099925
Error del epoch 19 : 0.7242419
Error del epoch 20 : 0.7167283
Error del epoch 21 : 0.7105941
Error del epoch 22 : 0.7080135
Error del epoch 23 : 0.7119852
Error del epoch 24 : 0.6886212
Error del epoch 25 : 0.6998307
Error del epoch 26 : 0.6867997
Error del epoch 27 : 0.7086529
Error del epoch 28 : 0.7064203
Error del epoch 29 : 0.694941
Error del epoch 30 : 0.6856667
Error del epoch 31 : 0.7003869
Error del epoch 32 :

## 1. Referencias


In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15141469021641961845
]


### Pregunta 2

In [6]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(32, input_dim=x_train.shape[1], activation="sigmoid"))
model.add(Dense(16, activation="sigmoid"))
model.add(Dense(3, activation="softmax"))
model.compile(optimizer=SGD(lr=0.1),loss="mse", metrics=["accuracy"])

In [7]:
model.fit(x_train, y_onehot_train, epochs=100, batch_size=1, verbose=1)

Epoch 1/100
100/100 [==============================] - 0s 3ms/step - loss: 0.2282 - acc: 0.3500
Epoch 2/100
100/100 [==============================] - 0s 652us/step - loss: 0.2169 - acc: 0.4300
Epoch 3/100
100/100 [==============================] - 0s 677us/step - loss: 0.2097 - acc: 0.3800
Epoch 4/100
100/100 [==============================] - 0s 662us/step - loss: 0.1985 - acc: 0.6100
Epoch 5/100
100/100 [==============================] - 0s 652us/step - loss: 0.1829 - acc: 0.6400
Epoch 6/100
100/100 [==============================] - 0s 712us/step - loss: 0.1635 - acc: 0.6800
Epoch 7/100
100/100 [==============================] - 0s 647us/step - loss: 0.1451 - acc: 0.7000
Epoch 8/100
100/100 [==============================] - 0s 652us/step - loss: 0.1323 - acc: 0.7300
Epoch 9/100
100/100 [==============================] - 0s 642us/step - loss: 0.1196 - acc: 0.8200
Epoch 10/100
100/100 [==============================] - 0s 667us/step - loss: 0.1091 - acc: 0.8200
Epoch 11/100
100/100 

100/100 [==============================] - 0s 657us/step - loss: 0.0143 - acc: 0.9800
Epoch 84/100
100/100 [==============================] - 0s 662us/step - loss: 0.0145 - acc: 0.9800
Epoch 85/100
100/100 [==============================] - 0s 647us/step - loss: 0.0148 - acc: 0.9700
Epoch 86/100
100/100 [==============================] - 0s 642us/step - loss: 0.0148 - acc: 0.9700
Epoch 87/100
100/100 [==============================] - 0s 642us/step - loss: 0.0143 - acc: 0.9800
Epoch 88/100
100/100 [==============================] - 0s 637us/step - loss: 0.0146 - acc: 0.9700
Epoch 89/100
100/100 [==============================] - 0s 642us/step - loss: 0.0124 - acc: 0.9700
Epoch 90/100
100/100 [==============================] - 0s 652us/step - loss: 0.0150 - acc: 0.9800
Epoch 91/100
100/100 [==============================] - 0s 672us/step - loss: 0.0136 - acc: 0.9600
Epoch 92/100
100/100 [==============================] - 0s 642us/step - loss: 0.0146 - acc: 0.9700
Epoch 93/100
100/100 [=